In [ ]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import base64
import pandas as pd 
import time
from email.mime.text import MIMEText
import email
import traceback

In [ ]:
#Autenticação do usuario e criação do arquivo tokens.json

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly', 
          'https://www.googleapis.com/auth/gmail.modify']


creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

In [ ]:
creds = Credentials.from_authorized_user_file('token.json', SCOPES)
service = build('gmail', 'v1', credentials=creds)

In [ ]:
#Função que escreve os emails em um arquivo txt
def get_emails():
    with open('emails.txt', 'w', encoding='utf-8') as f:
        try:
            service = build('gmail', 'v1', credentials=creds)
            results = service.users().messages().list(userId='me', maxResults=10).execute()
            messages = results.get('messages', [])

            for message in messages:
                msg = service.users().messages().get(userId='me', id=message['id']).execute()
                payload = msg['payload']
                headers = payload['headers']
                
                #Escreve o remetente, o assunto e a data
                for header in headers:
                    if header['name'] == 'From':
                        f.write(f"From: {header['value']}\n")
                    if header['name'] == 'Subject':
                        f.write(f"Subject: {header['value']}\n")
                    if header['name'] == 'Date':
                        f.write(f"Date: {header['value']}\n")
                
                #Se o corpo do email está vazio ou não segue o padrao MIME, a mensagem abaixo é escrita e o loop volta
                if 'parts' not in payload:
                    f.write("e-mail não contem nenhuma parte com o tipo MIME\n")
                    f.write("---------------------------------------------------------\n")    
                    continue
            
                #Escreve o corpo do email
                parts = payload['parts']
                for part in parts:
                    if part['mimeType'] == 'text/plain':
                        body = part['body']['data']
                        decoded_body = base64.urlsafe_b64decode(body).decode('utf-8')
                        f.write(f"Body: {decoded_body}\n")

                f.write("---------------------------------------------------------\n")
        except HttpError as error:
            f.write(f"An error occurred: {error}\n")
            messages = []

    return messages

get_emails()

In [ ]:
def searchStringMessage(service, userID, searchString):
    """
    Esta função irá retornar a lista com ID das mensagem que possuem a string (palavra) em questão
    Args:
        service: autenticação da API
        userID: id do usuário, no caso, o cliente, será sempre "me"(eu)
        searchString: apenas retornará as mensagem com o texto ou palavra presente presente
    Returns: lista com todos os IDs com a searchString em específico.
    """
    try: 
        idList = [] #inicializando a lista
        
        #adquirindo a mensagem (aonde a mágica acontece!)
        searchIds = service.users().messages().list(userId=userID, q=searchString).execute()
        
        try:
            EmailIds = searchIds ['messages']
        except KeyError:
            print("retornado 0 resultados")
            return idList
        
        if len(EmailIds)>=1:    #anexa todos os emails na lista "idList"
            for msg_id in EmailIds:
                idList.append(msg_id['id'])
            return (idList)
    except HttpError as error:
        print("ocorreu um erro: %s", error) 

In [ ]:
def viewMessage(service, userID, msgID):
    """
    Esta função mostra os emails presente na lista de ID
    Args:
        service: autenticação da API
        userID: id do usuário, no caso, o cliente, será sempre "me"(eu)
        msgID: iD da mensagem em questão
    Returns: o email em sua parte texto e html
    """
    try:
        message = service.users().messages().get(userId=userID, id=msgID, format='raw').execute() #retorna a mensagem em formato raw codificado em base64
        mime_msg = email.message_from_bytes(base64.urlsafe_b64decode(message['raw']))
        
        print("-----")        
        contentType = mime_msg.get_content_maintype()
        if contentType == 'multipart':
            for part in mime_msg.get_payload():
                if part.get_content_maintype() == 'text':
                    print(part.get_payload())
        elif contentType == 'text':
            print(mime_msg.get_payload())
        print("-----")
        
    except Exception as error:
        print("ocorreu um erro: %s", error)

In [ ]:
def viewMessageSnippet(service, userID, msgID):
    """
    Esta função mostra os emails presente na lista de ID, apenas o subject e o snippet da mensagem
    Args:
        service: autenticação da API
        userID: id do usuário, no caso, o cliente, será sempre "me"(eu)
        msgID: iD da mensagem em questão
    Returns: o snippet do email
    """
    try:
        message = service.users().messages().get(userId=userID, id=msgID, format='raw').execute() #retorna a mensagem em formato raw codificado em base64
        mime_msg = email.message_from_bytes(base64.urlsafe_b64decode(message['raw']))
        
        #print(mime_msg['from']) mostrar o email do remetente       
        #print(mime_msg['to'])  mostrar o email do destinatário
        print(mime_msg['subject'])
        print(message['snippet'])
        print(' ')
        
    except Exception as error:
        print("ocorreu um erro: %s", error) 


In [ ]:
userID = 'me'
words = ['REEMBOLSO', 'CONTA', 'ENTREGA', 'CONTATO', 'PAGAMENTO', 'FATURA', 'CANCELAMENTO', 'RASTREAR', 'NOTÍCIAS', 'OPINIÃO', 'HORA', 'PEDIDO', 'ORDEM']
IDs = {}


for word in words:
    IDs_palavras = searchStringMessage(service, userID, word)
    IDs[word] = IDs_palavras

#visualizar as mensagens de cada ID.
for word, ids in IDs.items():
    print(f'A palavra \'{word}\' tem {len(ids)} IDs correspondentes.')


for word, ids in IDs.items():
    print(word, ids)

In [ ]:
def searchLabel(labelList, labelName):
    """
    Esta função procura e compara se o label existe em uma lista ou não
    Args:
        labelList: lista para realizar a comparação
        labelName: nome a ser procurado na lista
    Returns: 0 para "achado" 1 para "não encontrado
    """        

    for i in labelList:
        if(str(labelName) == str(i)):
            return 0

    return 1

In [ ]:
created_label = {}
created_label_name = []

label_list = service.users().labels().list(userId=userID).execute()

size = len(label_list['labels'])
i = 0
labels = []#lista com o nome dos labels

while size > 0: #colocar o nome de todos os labels em uma lista
    label = label_list['labels'][i]['name']
    labels.append(label)
    size -= 1
    i += 1
    
print(labels)

for word in words:
    
    try: 
        label_name = word
        label = {'name': label_name, 'labelListVisibility': 'labelShow'}
        
        if(searchLabel(labels, label_name) == 1): #vai procurar na lista se já existe o nome do label atual
            created_label[label_name] = service.users().labels().create(userId='me', body=label).execute()
            print(f'Rótulo "{label_name}" criado com sucesso!')
            created_label_name.append(label_name)#chamada da API para criar o label
        else:
            print(f'Rótulo "{label_name}" já existe')
    
    except Exception as error:
        print("ocorreu um erro: ", error)

In [ ]:
print(created_label)
print(created_label_name)
label_id = []
for i in created_label_name:
    try:
        label_id.append(created_label[i]['id'])
   
    except Exception as error:
        print("ocorreu um erro", error)
        
print(label_id)

In [ ]:
i = 0

for word, ids in IDs.items():
    
    for label in created_label:
        if word == label: #apenas irá executar o trecho abaixo se o 'word' for um dos labels novos criados
            print(f'A palavra \'{word}\' tem {len(ids)} IDs correspondentes.')
            
            labelId = label_id[i]
            i += 1
            for message_id in ids:
                label_body = {
                    'removeLabelIds': ['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX'],
                    'addLabelIds': [labelId]
                    }
                service.users().messages().modify(userId='me', id=message_id, body=label_body).execute()

In [ ]:
labels = service.users().labels().list(userId='me').execute()
for label in labels['labels']:
    print(label['name'])